In [1]:
"""Uses Ray's RLLib to train agents to play Pistonball.

Author: Rohan (https://github.com/Rohan138)
"""

import os

#import supersuit as ss
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.tune.registry import register_env
from torch import nn

from pettingzoo.mpe import simple_tag_v2
from pettingzoo.test import render_test
from pettingzoo.test import performance_benchmark
from pettingzoo.test import test_save_obs

import time
import random

#raise NotImplementedError(
#    "There are currently bugs in this tutorial, we will fix them soon."
#)


C:\Users\rober\AppData\Roaming\Python\Python38\site-packages\pydot.py:17: UserWarning: `pydot` could not import `dot_parser`, so `pydot` will be unable to parse DOT files. The error was:  No module named 'pyparsing'
  warnings.warn(
C:\Users\rober\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
!pip install -U tensorflow-probability

  Using cached tensorflow_probability-0.19.0-py2.py3-none-any.whl (6.7 MB)


In [ ]:
!pip install --user --upgrade --force-reinstall tensorflow-probability

In [2]:

class CNNModelV2(TorchModelV2, nn.Module):
    def __init__(self, obs_space, act_space, num_outputs, *args, **kwargs):
        TorchModelV2.__init__(self, obs_space, act_space, num_outputs, *args, **kwargs)
        nn.Module.__init__(self)
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, [8, 8], stride=(4, 4)),
            nn.ReLU(),
            nn.Conv2d(32, 64, [4, 4], stride=(2, 2)),
            nn.ReLU(),
            nn.Conv2d(64, 64, [3, 3], stride=(1, 1)),
            nn.ReLU(),
            nn.Flatten(),
            (nn.Linear(3136, 512)),
            nn.ReLU(),
        )
        self.policy_fn = nn.Linear(512, num_outputs)
        self.value_fn = nn.Linear(512, 1)

    def forward(self, input_dict, state, seq_lens):
        model_out = self.model(input_dict["obs"].permute(0, 3, 1, 2))
        self._value_out = self.value_fn(model_out)
        return self.policy_fn(model_out), state

    def value_function(self):
        return self._value_out.flatten()


In [3]:


def env_creator(render_mode="rgb_array", cycles=200):
    
    from src.world import world_utils
    env = world_utils.env(render_mode=render_mode, max_cycles=cycles)

    return env


In [4]:
cycles = 800
env = env_creator(cycles=cycles)
env.reset()
current_cycle = 0
agent_count = 4  # todo: get from env
action_queue = []

for agent in env.agent_iter():
    if current_cycle >= cycles * agent_count:
        break
    if current_cycle % agent_count == 0:
        adversary_0_action = random.choice([0, 1, 2, 3, 4])
        adversary_1_action = random.choice([0, 1, 2, 3, 4])
        adversary_2_action = random.choice([0, 1, 2, 3, 4])
        good_agent_action = random.choice([0, 1, 2, 3, 4])

        action_queue += [
            adversary_0_action,
            adversary_1_action,
            adversary_2_action,
            good_agent_action
        ]
    # print(agent)
    env.render()
    # obs, reward, done, info = env.last()
    observation, cumulative_rewards, terminations, truncations, infos = env.last()
    action = action_queue.pop(0)
    env.step(action)
    current_cycle += 1

    # Following this but it's not working: https://github.com/openai/multiagent-particle-envs/issues/76
    # score+=reward
else:
    env.close()

In [8]:

if __name__ == "__main__":
    env_name = "pistonball_v6"

    register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))
    ModelCatalog.register_custom_model("CNNModelV2", CNNModelV2)

    config = (
        PPOConfig()
        .rollouts(num_rollout_workers=4, rollout_fragment_length=512)
        .training(
            train_batch_size=512,
            lr=2e-5,
            gamma=0.99,
            lambda_=0.9,
            use_gae=True,
            clip_param=0.4,
            grad_clip=None,
            entropy_coeff=0.1,
            vf_loss_coeff=0.25,
            sgd_minibatch_size=64,
            num_sgd_iter=10,
        )
        .environment(env=env_name, clip_actions=True)
        .debugging(log_level="ERROR")
        .framework(framework="torch")
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

    tune.run(
        "PPO",
        name="PPO",
        stop={"timesteps_total": 5000000},
        checkpoint_freq=10,
        local_dir="~/ray_results/" + env_name,
        config=config.to_dict(),
    )


ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.11; Detected an installation of version 2.8.0. Please upgrade TensorFlow to proceed.

In [5]:
os.environ.get("RLLIB_NUM_GPUS", "0")

'0'